In [1]:
from matplotlib import animation,rc
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from grid import *
from dep import *
import glob
import numpy as np

from IPython.display import HTML

In [2]:
rc('animation',html='html5')

In [14]:
PATH='/home/critechuser/COAST/EUR/20131101.00/'
path0='/home/critechuser/COAST/EUR/'

In [15]:
grid=Grid.fromfile(PATH+'eur.grd')
lon=grid.x[0,:].data
lat=grid.y[:,0].data

In [16]:
deb=Dep.read(PATH+'eur.dep',grid.shape)
b=deb.val
w=np.isnan(b)

Read data

In [24]:
data=Dataset(PATH+'trim-eur.nc')

In [25]:
tstart=datetime.datetime(2013,11,1,0)

In [26]:
h=data['S1'][:]
time=data['time'][:]
tw=[]
for l in time : tw.append(tstart+datetime.timedelta(0,int(l)))
tcw=np.array(tw).flatten()

In [43]:
iframes=h.shape[0]
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111)
bh=np.ma.masked_where(w==True,h[0,:,:].T)
#H=plt.imshow(np.flipud(h[0,:,:].T),animated=True, vmin=h.min(), vmax=h.max())
H=ax.imshow(np.flipud(bh),animated=True, vmin=h.min(), vmax=h.max())
plt.colorbar(H, orientation='horizontal')
date_text=fig.text(0.2,0.95, datetime.datetime.strftime(tcw[0],'%a %b %d  %H:%M:%S %Z %Y' ))

In [44]:
def animate(i):
    bh=np.ma.masked_where(w==True,h[i,:,:].T)
#    H.set_array(np.flipud(h[i,:,:].T))
    H.set_array(np.flipud(bh))
    date_text.set_text(datetime.datetime.strftime(tcw[i],'%a %b %d  %H:%M:%S %Z %Y' ))
    return H

In [45]:
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, frames=iframes, interval=200, blit=False, repeat=False)

In [46]:
#  display the animation
anim

In [14]:
#save animation
anim.save('tmp/webtest.mp4', fps=10, extra_args=['-vcodec','libx264'])

### Merging the 12h forecasts for longer time span

In [6]:
tstart=datetime.datetime(2016,9,28,0)

In [7]:
tend=datetime.datetime(2016,10,9,12)

In [8]:
dt=(tend-tstart).total_seconds()
ndt=dt/(3600*12)
ndt=np.int(ndt)+1

In [9]:
hcombined=[]
tw=[]

In [10]:
for it in range(ndt):
    idate=tstart+datetime.timedelta(hours=12*it)
    path=path0+'{}/{}/{}/{:02d}/'.format(idate.year,idate.month,idate.day,idate.hour)
    d=Dataset(path+'trim-hur.nc')
    hcombined.append(d['S1'][:12,:,:])
    time=d['time'][:12]
    tm =(idate-tstart).total_seconds()+(time-time[0])
    for l in tm : tw.append(tstart+datetime.timedelta(0,int(l)))
    

In [11]:
tcw=np.array(tw).flatten()

In [ ]:
hh=np.array(hcombined)

In [ ]:
hh=hh.reshape(tcw.size,w.shape[1],w.shape[0])

In [18]:
ha=np.transpose(hh,axes=(0,2,1))

In [19]:
iframes=ha.shape[0]
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111)
bh=np.ma.masked_where(w==True,ha[0,:,:])
#H=plt.imshow(np.flipud(h[0,:,:].T),animated=True, vmin=h.min(), vmax=h.max())
H=ax.imshow(np.flipud(bh),animated=True, vmin=hh.min(), vmax=hh.max())
plt.colorbar(H, orientation='horizontal')
date_text=ax.text(0.02,0.95, datetime.datetime.strftime(tcw[0],'%a %b %d  %H:%M:%S %Z %Y' ))

In [20]:
def animate(i):
    bh=np.ma.masked_where(w==True,ha[i,:,:])
#    H.set_array(np.flipud(ha[i,:,:]))
    H.set_array(np.flipud(bh))
    date_text.set_text(datetime.datetime.strftime(tcw[i],'%a %b %d  %H:%M:%S %Z %Y' ))
    return H

In [21]:
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, frames=iframes, interval=200, blit=False, repeat=False)

In [ ]:
#  display the animation
anim

In [ ]:
#save animation
anim.save('tmp/matthew1.mp4', fps=10, extra_args=['-vcodec','libx264'])